In [1]:
#data challenge yudha

### Importing Library

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Import the Data

In [3]:
#read employees data
dtypes_employees = {'employe_id': int, 'branch_id': int, 'salary': int} #, 'join_date': str, 'resign_date': str}
#parse_dates_employees = ['join_date', 'resign_date']
employees = pd.read_csv('employees.csv', dtype=dtypes_employees, usecols=['employe_id', 'branch_id', 'salary']) #parse_dates=parse_dates_employees,

#read timesheets data
dtypes_timesheets = {'timesheet_id': int, 'employee_id': int, 'date': str, 'checkin': str, 'checkout': str}
parse_dates_timesheets = ['date', 'checkin', 'checkout']
timesheets = pd.read_csv('timesheets.csv', dtype=dtypes_timesheets, parse_dates=parse_dates_timesheets)

In [4]:
#print(employees.info())
#print(timesheets.info())
#employees.describe()
#timesheets.describe()

#### preprocessing data

In [5]:
#check duplicated
print(employees.duplicated(subset=['employe_id']).any())
print(timesheets.duplicated().any())

True
False


In [6]:
#see the list of duplicated data
employees[employees.duplicated(subset=['employe_id'], keep=False)]

,employe_id,branch_id,salary
163,218078,1,10500000
164,218078,1,13000000


In [7]:
#remove duplicate and keep the latest one
employees = employees.drop_duplicates(subset=['employe_id'], keep='last')

In [22]:
#handling missing value on checkout fields, approach is fill with the average time
#timesheets['checkout'].fillna(timesheets.checkout.mean(), inplace=True)

#drop null values on checkout
timesheets.dropna(subset=['checkout'],inplace=True)

In [9]:
#create new feature, working_hours as checkin and checkout difference
timesheets['working_hours'] = (timesheets.checkout - timesheets.checkin).astype('timedelta64[h]')

In [36]:
#merge employee and timesheet data
merge_df = employees.merge(timesheets, left_on=['employe_id'], right_on=['employee_id'], how='left')

#### create the asked fields

In [11]:
merge_df['date'] = merge_df['date'].dt.to_period('M')
df_added_field = merge_df.groupby(['date','branch_id'], as_index=False)[["salary", "working_hours"]].sum()
df_added_field['year'] = df_added_field['date'].dt.year
df_added_field['month'] = df_added_field['date'].dt.month
df_added_field['salary_per_hour'] = df_added_field['salary']/df_added_field['working_hours']
final = df_added_field[['year', 'month', 'branch_id', 'salary_per_hour']].sort_values(by=['year', 'month', 'branch_id'])

In [35]:
def add_salary_per_hour(df):
    df['date'] = df['date'].dt.to_period('M')
    df_added_field = df.groupby(['date','branch_id'], as_index=False)[["salary", "working_hours"]].sum()
    df_added_field['year'] = df_added_field['date'].dt.year
    df_added_field['month'] = df_added_field['date'].dt.month
    df_added_field['salary_per_hour'] = df_added_field['salary']/df_added_field['working_hours']
    final = df_added_field[['year', 'month', 'branch_id', 'salary_per_hour']].sort_values(by=['year', 'month', 'branch_id'])
    return final

In [12]:
#df_added_field = merge_df.groupby(['date','branch_id'], as_index=False)[["salary", "working_hours"]].sum()
#df_added_field.sort_values(by=['date', 'branch_id'])

In [ ]:
salary_per_hour.append(final, ignore_index = True)